# Acknowledgement:
I looked at this youtube video for model architecture and training code: https://www.youtube.com/watch?v=Q0vvh95wes8

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import Javascript  # Restrict height of output cell.
from sklearn.metrics import ConfusionMatrixDisplay

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils import data

import jax
import jax.numpy as jnp
import flax
import flax.linen as nn
import optax
from jax.tree_util import tree_map
from flax.training import train_state

In [2]:
from utils import image_to_numpy
from utils import numpy_collate

In [3]:
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [19]:
# model saving
from flax import serialization
from flax.training import train_state

## Initialization

In [5]:
IMAGE_SIZE = 32
BATCH_SIZE = 128
DATA_MEANS = np.array([0.49139968, 0.48215841, 0.44653091])
DATA_STD = np.array([0.24703223, 0.24348513, 0.26158784])
CROP_SCALES = (0.8, 1.0)
CROP_RATIO = (0.9, 1.1)

SEED = 42

plt.style.use('dark_background')

In [6]:
class CNN(nn.Module):
  """A simple CNN model."""

  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten
    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)
    return x
     

## Load CIFAR 10 Dataset

In [7]:
classes = ('airplane, automobile, ship, truck', 'bird, dog, frog, horse')
# images in the test set will only be converted into numpy arrays
test_transform = image_to_numpy
# images in the train set will be randomly flipped, cropped, and then converted to numpy arrays
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE), scale=CROP_SCALES, ratio=CROP_RATIO),
    image_to_numpy
])

# Validation set should not use train_transform.
train_dataset = torchvision.datasets.CIFAR10('data', train=True, transform=train_transform, download=True)
val_dataset = torchvision.datasets.CIFAR10('data', train=True, transform=test_transform, download=True)
# We're going to splite the train and test sets by seed 0 since we're exploring the differences between S_init and S_batch
train_set, _ = torch.utils.data.random_split(train_dataset, [45000, 5000], generator=torch.Generator().manual_seed(0))
_, val_set = torch.utils.data.random_split(val_dataset, [45000, 5000], generator=torch.Generator().manual_seed(0))
test_set = torchvision.datasets.CIFAR10('data', train=False, transform=test_transform, download=True)

train_data_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=2, persistent_workers=True, collate_fn=numpy_collate,
)
val_data_loader = torch.utils.data.DataLoader(
    val_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, num_workers=2, persistent_workers=True, collate_fn=numpy_collate,
)
test_data_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, num_workers=2, persistent_workers=True, collate_fn=numpy_collate,
)
     

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [8]:
print('CHECK CHECK CHECK')
print(f'number of samples in train_set:         {len(train_set)}')
print(f'number of batches in train_data_loader: {len(train_data_loader)}')
print(f'number of samples / batch size:         {len(train_set)} / {BATCH_SIZE} = {len(train_set)/BATCH_SIZE}')
print(f'number of samples in test_set:          {len(test_set)}')
print(f'number of batches in test_data_loader:  {len(test_data_loader)}')
print(f'number of samples / batch size:         {len(test_set)} / {BATCH_SIZE} = {len(test_set)/BATCH_SIZE}')

CHECK CHECK CHECK
number of samples in train_set:         45000
number of batches in train_data_loader: 351
number of samples / batch size:         45000 / 128 = 351.5625
number of samples in test_set:          10000
number of batches in test_data_loader:  79
number of samples / batch size:         10000 / 128 = 78.125


In [9]:
print(f'size of images in the first train batch: {next(iter(train_data_loader))[0].shape}')
print(f'type of images in the first train batch: {next(iter(train_data_loader))[0].dtype}')
print(f'size of labels in the first train batch: {next(iter(train_data_loader))[1].shape}')
print(f'type of labels in the first train batch: {next(iter(train_data_loader))[1].dtype}')

size of images in the first train batch: (128, 32, 32, 3)
type of images in the first train batch: float64
size of labels in the first train batch: (128,)
type of labels in the first train batch: int64


## Initializing the Model (RANDOM SEEDS)

In [10]:
model = CNN()

In [11]:
# Thi
optimizer = optax.adam(learning_rate=1e-4)

# inp_rng affects the batches
# init_rng affects the initialization of the model
rng, inp_rng, init_rng = jax.random.split(jax.random.PRNGKey(SEED), 3)
params = model.init(jax.random.PRNGKey(SEED),
                    jax.random.normal(inp_rng, (BATCH_SIZE, 32, 32, 3)))

model_state = train_state.TrainState.create(apply_fn=model.apply,
                                            params=params,
                                            tx=optimizer)


## Training

In [12]:
@jax.jit
def apply_model(state, images, labels):
  """Computes gradients, loss and accuracy for a single batch."""

  def loss_fn(params):
    logits = state.apply_fn(params, images)
    one_hot = jax.nn.one_hot(labels, logits.shape[1])
    loss = jnp.mean(optax.softmax_cross_entropy(logits=logits, labels=one_hot))
    return loss, logits

  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(state.params)
  accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
  return grads, loss, accuracy


@jax.jit
def update_model(state, grads):
  return state.apply_gradients(grads=grads)

In [13]:
writer = SummaryWriter('runs/experiment_1')

def train_epoch(state, data_loader, current_epoch):
    """Train for a single epoch."""
    epoch_loss = []
    epoch_accuracy = []

    progress_bar = tqdm(data_loader, desc="Training")
    
    for batch in progress_bar:
        batch_images, batch_labels = batch
        grads, loss, accuracy = apply_model(state, batch_images, batch_labels)
        state = update_model(state, grads)
        
        epoch_loss.append(loss)
        epoch_accuracy.append(accuracy)

        # Update progress bar with current loss and accuracy
        progress_bar.set_postfix(loss=np.mean(epoch_loss), accuracy=np.mean(epoch_accuracy))

    train_loss = np.mean(epoch_loss)
    train_accuracy = np.mean(epoch_accuracy)

    # Log metrics to TensorBoard
    writer.add_scalar('Loss/train', train_loss, current_epoch)  # Use the current_epoch variable
    writer.add_scalar('Accuracy/train', train_accuracy, current_epoch)

    return state, train_loss, train_accuracy

In [14]:
def train_model(state, train_data_loader, num_epochs):
    # Training loop
    for epoch in range(num_epochs):
        print(f'Starting epoch {epoch + 1}/{num_epochs}')
        
        # Call the train_epoch function, passing the current epoch
        state, train_loss, train_accuracy = train_epoch(state, train_data_loader, epoch)

        # Print training loss and accuracy
        print(f'epoch: {epoch + 1:03d}, train loss: {train_loss:.4f}, train accuracy: {train_accuracy:.4f}')
    
    return state


In [15]:
trained_model_state = train_model(model_state, train_data_loader, num_epochs=35)
writer.close()

Starting epoch 1/35


Training: 100%|████| 351/351 [00:15<00:00, 22.05it/s, accuracy=0.402, loss=1.69]


epoch: 001, train loss: 1.6871, train accuracy: 0.4016
Starting epoch 2/35


Training: 100%|████| 351/351 [00:15<00:00, 22.00it/s, accuracy=0.496, loss=1.43]


epoch: 002, train loss: 1.4269, train accuracy: 0.4965
Starting epoch 3/35


...

epoch: 033, train loss: 0.7483, train accuracy: 0.7422
Starting epoch 34/35


Training: 100%|███| 351/351 [00:17<00:00, 20.14it/s, accuracy=0.744, loss=0.742]


epoch: 034, train loss: 0.7417, train accuracy: 0.7436
Starting epoch 35/35


Training: 100%|███| 351/351 [00:17<00:00, 20.28it/s, accuracy=0.747, loss=0.732]

epoch: 035, train loss: 0.7322, train accuracy: 0.7472


# Testing

In [16]:
test_loss = []
test_accuracy = []

for batch in test_data_loader:
  batch_images, batch_labels = batch
  _, loss, accuracy = apply_model(trained_model_state, batch_images, batch_labels)
  test_loss.append(loss)
  test_accuracy.append(accuracy)

print(f'loss: {np.mean(test_loss):.4f}, accuracy: {np.mean(test_accuracy):.4f}')

loss: 0.8824, accuracy: 0.6957


## Saving/Loading Model

In [17]:
# Saving model
with open('trained_model.pkl','wb') as f:
    f.write(serialization.to_bytes(trained_model_state))

In [21]:
# Loading the model
with open('trained_model.pkl', 'rb') as f:
    loaded_model_state = serialization.from_bytes(train_state.TrainState, f.read())

In [22]:
loaded_model_state

{'step': array(12285, dtype=int32),
 'params': {'params': {'Conv_0': {'bias': array([-0.05509524, -0.01494891,  0.02423832,  0.17565621,  0.03038183,
            0.02479529, -0.02108995, -0.06242558,  0.12463821,  0.10383037,
            0.10325531,  0.06030441, -0.00580891, -0.07965636,  0.09768064,
            0.12192987, -0.10313437,  0.03542982, -0.05132829,  0.10119658,
            0.00237998, -0.01142217,  0.17714292,  0.0493258 , -0.02489948,
            0.11390908, -0.01538698,  0.12576143,  0.08806267,  0.00765791,
            0.12520294,  0.01378171], dtype=float32),
    'kernel': array([[[[ 1.84198797e-01, -4.88828160e-02, -3.60032320e-01,
               1.28478050e-01, -1.18141524e-01,  6.02924787e-02,
              -1.40627488e-01,  1.87220395e-01,  2.35586509e-01,
              -2.42053360e-01,  3.93796340e-02,  3.61317307e-01,
              -1.21444546e-01, -7.78316185e-02,  2.35576987e-01,
               1.25078216e-01,  1.38448432e-01, -1.73388034e-01,
              -3